In [95]:
import numpy as np
import imageio
import scipy as sio
import os
import pandas as pd
from scipy import ndimage

In [96]:
#%%
from mlxtend.data import loadlocal_mnist
import platform

if not platform.system() == 'Windows':
    X_data, Y_label = loadlocal_mnist(
            images_path='MNIST/train-images-idx3-ubyte',
            labels_path='MNIST/train-labels-idx1-ubyte')

else:
    X_data, Y_label = loadlocal_mnist(
            images_path='MNIST/train-images.idx3-ubyte',
            labels_path='MNIST/train-labels.idx1-ubyte')

## Testing and Screenshots

In [98]:
import matplotlib.pyplot as plt

from skimage.io import imread
from skimage import data_dir
from skimage.transform import radon, iradon
from skimage.transform import resize
from scipy.ndimage import zoom
from numpy import asarray

from skimage.filters import threshold_mean

X_data = X_data.astype('float32')

P_NAngle=4
Downsampling=28

df=pd.DataFrame([])

for i in range(60000):
    P_Angle=0
    RBC=np.array([])
    while P_Angle <180:
        data=np.array([])
        Th_P=radon(X_data[i].reshape(Downsampling,Downsampling),theta=[P_Angle])
    #Creating Threshold Values
        thresh = threshold_mean(Th_P)
        binary = Th_P >= thresh
        data = np.asarray(binary)
        RBC=np.append(RBC,data)
        P_Angle=P_Angle+(180/P_NAngle)
        RBC=np.expand_dims(RBC,axis=0)
    new_row={'Labels':Y_label[i],'Barcodes':RBC}
    #df=df.append(new_row,ignore_index=True)
    #plt.axis('off')
    #plt.imshow(RBC, cmap='gray',interpolation='nearest')
    #print(RBC)
    #plt.savefig('test1.jpg',bbox_inches='tight',pad_inches=0)
#df.to_excel('MNISTBarcodeDataset.xlsx',index=False)
print ("done")
#%%

done


## Final: MNIST Radon Barcode Generation Algorithm

In [99]:
import matplotlib.pyplot as plt

from skimage.io import imread
from skimage import data_dir
from skimage.transform import radon, iradon
from skimage.transform import resize
from scipy.ndimage import zoom
from numpy import asarray

from skimage.filters import threshold_mean

X_data = X_data.astype('float32')

P_NAngle=4
Downsampling=28

Hamming=[]

for i in range(60000):
    P_Angle=0
    RBC=np.array([])
    while P_Angle <180:
        data=np.array([])
#Radon Projections
        Th_P=radon(X_data[i].reshape(Downsampling,Downsampling),theta=[P_Angle])
#Creating Threshold Values
        thresh = threshold_mean(Th_P)#Finds the threshold mean of the projection
        binary = Th_P >= thresh#Determines whether 1 or 0 based on relation to the threshold mean
        data = np.asarray(binary)
        RBC=np.append(RBC,data)#Adds to the full barcode, till all projections are binarized 
        P_Angle=P_Angle+(180/P_NAngle)#Changes the angle of radon projection
        RBC=np.expand_dims(RBC,axis=0)
    Hamming.append(RBC)
print ("Barcodes Generated")
#%%

Barcodes Generated


## Final: MNIST Radon Barcode Searching Algorithm

In [84]:
from scipy.spatial.distance import hamming
import random
import timeit

potential=1
value=0
potential_label=0
count=0
sum=0
for i in range(100):
    x=random.randint(1,59999)
    print('The testing label is: ',Y_label[x],' and the image number is: ',x)
    potential=1
    value=0
    potential_label=0
    sum=sum+1
    for j in range(10000):
        value=hamming(Hamming[x],Hamming[j])
        if (value<potential and x!=j):
            potential=value
            potential_label=j
            print('The potential label is: ', Y_label[j],' and the image number is: ',j,' and the Hamming value is ',potential)
    if (Y_label[potential_label]==Y_label[x]):
        count=count+1
        average=count/sum
print('The average of the dataset will be ',average*100,'%')

The testing label is:  2  and the image number is:  50295
The potential label is:  5  and the image number is:  0  and the Hamming value is  0.2857142857142857
The potential label is:  0  and the image number is:  1  and the Hamming value is  0.23214285714285715
The potential label is:  2  and the image number is:  16  and the Hamming value is  0.1875
The potential label is:  2  and the image number is:  82  and the Hamming value is  0.17857142857142858
The potential label is:  2  and the image number is:  178  and the Hamming value is  0.16964285714285715
The potential label is:  2  and the image number is:  187  and the Hamming value is  0.16071428571428573
The potential label is:  2  and the image number is:  213  and the Hamming value is  0.13392857142857142
The potential label is:  2  and the image number is:  325  and the Hamming value is  0.125
The potential label is:  2  and the image number is:  375  and the Hamming value is  0.10714285714285714
The potential label is:  2  and

The potential label is:  0  and the image number is:  7669  and the Hamming value is  0.008928571428571428
The testing label is:  0  and the image number is:  46627
The potential label is:  5  and the image number is:  0  and the Hamming value is  0.24107142857142858
The potential label is:  0  and the image number is:  1  and the Hamming value is  0.0625
The potential label is:  0  and the image number is:  37  and the Hamming value is  0.05357142857142857
The potential label is:  8  and the image number is:  225  and the Hamming value is  0.03571428571428571
The potential label is:  0  and the image number is:  427  and the Hamming value is  0.026785714285714284
The potential label is:  8  and the image number is:  1319  and the Hamming value is  0.017857142857142856
The potential label is:  0  and the image number is:  1495  and the Hamming value is  0.008928571428571428
The testing label is:  4  and the image number is:  52377
The potential label is:  5  and the image number is:  0

The potential label is:  1  and the image number is:  978  and the Hamming value is  0.017857142857142856
The testing label is:  0  and the image number is:  47752
The potential label is:  5  and the image number is:  0  and the Hamming value is  0.2767857142857143
The potential label is:  0  and the image number is:  1  and the Hamming value is  0.09821428571428571
The potential label is:  0  and the image number is:  37  and the Hamming value is  0.08928571428571429
The potential label is:  0  and the image number is:  56  and the Hamming value is  0.08035714285714286
The potential label is:  0  and the image number is:  88  and the Hamming value is  0.07142857142857142
The potential label is:  0  and the image number is:  114  and the Hamming value is  0.05357142857142857
The potential label is:  0  and the image number is:  156  and the Hamming value is  0.044642857142857144
The potential label is:  0  and the image number is:  427  and the Hamming value is  0.026785714285714284
Th

The potential label is:  2  and the image number is:  6063  and the Hamming value is  0.044642857142857144
The testing label is:  8  and the image number is:  41013
The potential label is:  5  and the image number is:  0  and the Hamming value is  0.24107142857142858
The potential label is:  0  and the image number is:  1  and the Hamming value is  0.1875
The potential label is:  1  and the image number is:  6  and the Hamming value is  0.10714285714285714
The potential label is:  8  and the image number is:  348  and the Hamming value is  0.09821428571428571
The potential label is:  0  and the image number is:  668  and the Hamming value is  0.08928571428571429
The potential label is:  8  and the image number is:  726  and the Hamming value is  0.08035714285714286
The potential label is:  8  and the image number is:  1254  and the Hamming value is  0.07142857142857142
The potential label is:  8  and the image number is:  7441  and the Hamming value is  0.0625
The testing label is:  9 

The potential label is:  4  and the image number is:  5033  and the Hamming value is  0.03571428571428571
The potential label is:  4  and the image number is:  5091  and the Hamming value is  0.026785714285714284
The testing label is:  8  and the image number is:  8701
The potential label is:  5  and the image number is:  0  and the Hamming value is  0.25
The potential label is:  0  and the image number is:  1  and the Hamming value is  0.19642857142857142
The potential label is:  9  and the image number is:  4  and the Hamming value is  0.13392857142857142
The potential label is:  7  and the image number is:  29  and the Hamming value is  0.09821428571428571
The potential label is:  4  and the image number is:  2484  and the Hamming value is  0.08928571428571429
The potential label is:  0  and the image number is:  8449  and the Hamming value is  0.08035714285714286
The testing label is:  7  and the image number is:  15202
The potential label is:  5  and the image number is:  0  and t

The potential label is:  2  and the image number is:  8864  and the Hamming value is  0.0625
The testing label is:  0  and the image number is:  31836
The potential label is:  5  and the image number is:  0  and the Hamming value is  0.22321428571428573
The potential label is:  0  and the image number is:  1  and the Hamming value is  0.0625
The potential label is:  0  and the image number is:  69  and the Hamming value is  0.05357142857142857
The potential label is:  0  and the image number is:  118  and the Hamming value is  0.03571428571428571
The potential label is:  0  and the image number is:  303  and the Hamming value is  0.026785714285714284
The potential label is:  0  and the image number is:  1709  and the Hamming value is  0.017857142857142856
The potential label is:  0  and the image number is:  3160  and the Hamming value is  0.008928571428571428
The testing label is:  1  and the image number is:  32246
The potential label is:  5  and the image number is:  0  and the Hamm

The testing label is:  4  and the image number is:  7459
The potential label is:  5  and the image number is:  0  and the Hamming value is  0.22321428571428573
The potential label is:  9  and the image number is:  4  and the Hamming value is  0.07142857142857142
The potential label is:  9  and the image number is:  45  and the Hamming value is  0.05357142857142857
The potential label is:  9  and the image number is:  153  and the Hamming value is  0.03571428571428571
The potential label is:  4  and the image number is:  4567  and the Hamming value is  0.026785714285714284
The potential label is:  4  and the image number is:  7487  and the Hamming value is  0.008928571428571428
The testing label is:  9  and the image number is:  55835
The potential label is:  5  and the image number is:  0  and the Hamming value is  0.23214285714285715
The potential label is:  0  and the image number is:  1  and the Hamming value is  0.19642857142857142
The potential label is:  9  and the image number i

The testing label is:  7  and the image number is:  5626
The potential label is:  5  and the image number is:  0  and the Hamming value is  0.19642857142857142
The potential label is:  0  and the image number is:  1  and the Hamming value is  0.17857142857142858
The potential label is:  9  and the image number is:  4  and the Hamming value is  0.0625
The potential label is:  9  and the image number is:  687  and the Hamming value is  0.05357142857142857
The potential label is:  9  and the image number is:  5371  and the Hamming value is  0.044642857142857144
The testing label is:  3  and the image number is:  23877
The potential label is:  5  and the image number is:  0  and the Hamming value is  0.23214285714285715
The potential label is:  1  and the image number is:  3  and the Hamming value is  0.21428571428571427
The potential label is:  9  and the image number is:  4  and the Hamming value is  0.15178571428571427
The potential label is:  7  and the image number is:  38  and the Ha

The testing label is:  3  and the image number is:  17997
The potential label is:  5  and the image number is:  0  and the Hamming value is  0.20535714285714285
The potential label is:  3  and the image number is:  7  and the Hamming value is  0.125
The potential label is:  3  and the image number is:  157  and the Hamming value is  0.10714285714285714
The potential label is:  3  and the image number is:  281  and the Hamming value is  0.09821428571428571
The potential label is:  3  and the image number is:  1603  and the Hamming value is  0.08035714285714286
The potential label is:  3  and the image number is:  7123  and the Hamming value is  0.0625
The testing label is:  9  and the image number is:  6368
The potential label is:  5  and the image number is:  0  and the Hamming value is  0.25892857142857145
The potential label is:  9  and the image number is:  4  and the Hamming value is  0.10714285714285714
The potential label is:  7  and the image number is:  15  and the Hamming valu

The testing label is:  1  and the image number is:  24366
The potential label is:  5  and the image number is:  0  and the Hamming value is  0.25892857142857145
The potential label is:  0  and the image number is:  1  and the Hamming value is  0.1875
The potential label is:  1  and the image number is:  3  and the Hamming value is  0.16964285714285715
The potential label is:  1  and the image number is:  6  and the Hamming value is  0.10714285714285714
The potential label is:  1  and the image number is:  8  and the Hamming value is  0.08928571428571429
The potential label is:  1  and the image number is:  14  and the Hamming value is  0.08035714285714286
The potential label is:  1  and the image number is:  40  and the Hamming value is  0.026785714285714284
The potential label is:  1  and the image number is:  2906  and the Hamming value is  0.008928571428571428
The testing label is:  0  and the image number is:  4744
The potential label is:  5  and the image number is:  0  and the Ha

In [111]:
import matplotlib.pyplot as plt

from skimage.io import imread
from skimage import data_dir
from skimage.transform import radon, iradon
from skimage.transform import resize
from scipy.ndimage import zoom
from numpy import asarray

from skimage.filters import threshold_mean
from collections import defaultdict
X_data = X_data.astype('float32')

P_NAngle=4
Downsampling=28

Hamming2=defaultdict(list)

for i in range(60000):
    P_Angle=0
    RBC=np.array([])
    Hamming2[i]=[]
    while P_Angle <180:
        data=np.array([])
#Radon Projections
        Th_P=radon(X_data[i].reshape(Downsampling,Downsampling),theta=[P_Angle])
#Creating Threshold Values
        thresh = threshold_mean(Th_P)#Finds the threshold mean of the projection
        binary = Th_P >= thresh#Determines whether 1 or 0 based on relation to the threshold mean
        data = np.asarray(binary)
        RBC=np.append(RBC,data)#Adds to the full barcode, till all projections are binarized 
        P_Angle=P_Angle+(180/P_NAngle)#Changes the angle of radon projection
        RBC=np.expand_dims(RBC,axis=0)
    Hamming2[i].append(RBC)
print ("Barcodes Generated")
#%%

Barcodes Generated


In [123]:
from scipy.spatial.distance import hamming
import random
import timeit

potential=1
value=0
potential_label=0
count=0
sum=0
for i in range(100):
    x=random.randint(1,59999)
    print('The testing label is: ',Y_label[x],' and the image number is: ',x)
    potential=1
    value=0
    potential_label=0
    sum=sum+1
    for j in range(2000):
        value=hamming(Hamming2[x],Hamming2[j])
        if (value<potential and x!=j):
            potential=value
            potential_label=j
            print('The potential label is: ', Y_label[j],' and the image number is: ',j,' and the Hamming value is ',potential)
    if (Y_label[potential_label]==Y_label[x]):
        count=count+1
        average=count/sum
print('The average of the dataset will be ',average*100,'%')

The testing label is:  5  and the image number is:  53054
The potential label is:  5  and the image number is:  0  and the Hamming value is  0.16964285714285715
The potential label is:  1  and the image number is:  24  and the Hamming value is  0.15178571428571427
The potential label is:  1  and the image number is:  70  and the Hamming value is  0.13392857142857142
The potential label is:  2  and the image number is:  318  and the Hamming value is  0.125
The potential label is:  9  and the image number is:  864  and the Hamming value is  0.10714285714285714
The testing label is:  1  and the image number is:  46173
The potential label is:  5  and the image number is:  0  and the Hamming value is  0.22321428571428573
The potential label is:  0  and the image number is:  1  and the Hamming value is  0.15178571428571427
The potential label is:  1  and the image number is:  3  and the Hamming value is  0.11607142857142858
The potential label is:  1  and the image number is:  6  and the Ham

The potential label is:  1  and the image number is:  1860  and the Hamming value is  0.03571428571428571
The testing label is:  6  and the image number is:  45479
The potential label is:  5  and the image number is:  0  and the Hamming value is  0.23214285714285715
The potential label is:  1  and the image number is:  3  and the Hamming value is  0.21428571428571427
The potential label is:  9  and the image number is:  4  and the Hamming value is  0.15178571428571427
The potential label is:  4  and the image number is:  9  and the Hamming value is  0.14285714285714285
The potential label is:  6  and the image number is:  13  and the Hamming value is  0.08928571428571429
The potential label is:  6  and the image number is:  36  and the Hamming value is  0.08035714285714286
The potential label is:  6  and the image number is:  66  and the Hamming value is  0.044642857142857144
The potential label is:  6  and the image number is:  1041  and the Hamming value is  0.03571428571428571
The t

The potential label is:  5  and the image number is:  0  and the Hamming value is  0.21428571428571427
The potential label is:  0  and the image number is:  1  and the Hamming value is  0.14285714285714285
The potential label is:  2  and the image number is:  16  and the Hamming value is  0.11607142857142858
The potential label is:  2  and the image number is:  82  and the Hamming value is  0.08928571428571429
The potential label is:  2  and the image number is:  591  and the Hamming value is  0.08035714285714286
The testing label is:  8  and the image number is:  10444
The potential label is:  5  and the image number is:  0  and the Hamming value is  0.17857142857142858
The potential label is:  9  and the image number is:  4  and the Hamming value is  0.15178571428571427
The potential label is:  1  and the image number is:  6  and the Hamming value is  0.11607142857142858
The potential label is:  8  and the image number is:  144  and the Hamming value is  0.10714285714285714
The poten

The potential label is:  6  and the image number is:  735  and the Hamming value is  0.07142857142857142
The testing label is:  2  and the image number is:  20960
The potential label is:  5  and the image number is:  0  and the Hamming value is  0.20535714285714285
The potential label is:  3  and the image number is:  7  and the Hamming value is  0.17857142857142858
The potential label is:  2  and the image number is:  16  and the Hamming value is  0.14285714285714285
The potential label is:  2  and the image number is:  82  and the Hamming value is  0.11607142857142858
The potential label is:  2  and the image number is:  866  and the Hamming value is  0.10714285714285714
The testing label is:  6  and the image number is:  34060
The potential label is:  5  and the image number is:  0  and the Hamming value is  0.22321428571428573
The potential label is:  0  and the image number is:  1  and the Hamming value is  0.15178571428571427
The potential label is:  4  and the image number is:  

The testing label is:  3  and the image number is:  51343
The potential label is:  5  and the image number is:  0  and the Hamming value is  0.25892857142857145
The potential label is:  0  and the image number is:  1  and the Hamming value is  0.16964285714285715
The potential label is:  0  and the image number is:  21  and the Hamming value is  0.15178571428571427
The potential label is:  8  and the image number is:  146  and the Hamming value is  0.14285714285714285
The potential label is:  8  and the image number is:  295  and the Hamming value is  0.13392857142857142
The potential label is:  3  and the image number is:  1651  and the Hamming value is  0.125
The potential label is:  3  and the image number is:  1661  and the Hamming value is  0.08928571428571429
The testing label is:  8  and the image number is:  6774
The potential label is:  5  and the image number is:  0  and the Hamming value is  0.22321428571428573
The potential label is:  0  and the image number is:  1  and the

The potential label is:  4  and the image number is:  20  and the Hamming value is  0.15178571428571427
The potential label is:  4  and the image number is:  60  and the Hamming value is  0.08928571428571429
The potential label is:  4  and the image number is:  380  and the Hamming value is  0.08035714285714286
The testing label is:  8  and the image number is:  57379
The potential label is:  5  and the image number is:  0  and the Hamming value is  0.21428571428571427
The potential label is:  0  and the image number is:  1  and the Hamming value is  0.07142857142857142
The potential label is:  0  and the image number is:  37  and the Hamming value is  0.0625
The potential label is:  0  and the image number is:  81  and the Hamming value is  0.044642857142857144
The potential label is:  8  and the image number is:  245  and the Hamming value is  0.017857142857142856
The testing label is:  7  and the image number is:  30558
The potential label is:  5  and the image number is:  0  and th

The testing label is:  8  and the image number is:  48280
The potential label is:  5  and the image number is:  0  and the Hamming value is  0.2857142857142857
The potential label is:  0  and the image number is:  1  and the Hamming value is  0.23214285714285715
The potential label is:  1  and the image number is:  6  and the Hamming value is  0.22321428571428573
The potential label is:  6  and the image number is:  18  and the Hamming value is  0.21428571428571427
The potential label is:  2  and the image number is:  25  and the Hamming value is  0.1875
The potential label is:  0  and the image number is:  56  and the Hamming value is  0.17857142857142858
The potential label is:  0  and the image number is:  88  and the Hamming value is  0.16964285714285715
The potential label is:  6  and the image number is:  126  and the Hamming value is  0.16071428571428573
The potential label is:  1  and the image number is:  134  and the Hamming value is  0.14285714285714285
The potential label i

The testing label is:  6  and the image number is:  58438
The potential label is:  5  and the image number is:  0  and the Hamming value is  0.24107142857142858
The potential label is:  0  and the image number is:  1  and the Hamming value is  0.1875
The potential label is:  1  and the image number is:  6  and the Hamming value is  0.14285714285714285
The potential label is:  6  and the image number is:  13  and the Hamming value is  0.11607142857142858
The potential label is:  6  and the image number is:  73  and the Hamming value is  0.10714285714285714
The potential label is:  6  and the image number is:  83  and the Hamming value is  0.09821428571428571
The potential label is:  6  and the image number is:  221  and the Hamming value is  0.08928571428571429
The potential label is:  6  and the image number is:  286  and the Hamming value is  0.07142857142857142
The testing label is:  2  and the image number is:  23814
The potential label is:  5  and the image number is:  0  and the H